In [111]:
!ls ../../evaluations/hate_speech/task_b/no_best

beto.json


In [118]:
import json
import pandas as pd

evaluations = {

}

for model_name, path in [
    ('beto', '../../evaluations/hate_speech/task_b/beto.json'),
    ('beto-hier', '../../evaluations/hate_speech/task_b/beto-hierarchical-gamma-0.1.json'),
    ('beto-hier-zero', '../../evaluations/hate_speech/task_b/beto-hierarchical-gamma-0.json'),
   
    ('robertuito', '../../evaluations/hate_speech/task_b/robertuito.json'),
    ('robertuito-hier', '../../evaluations/hate_speech/task_b/robertuito-hierarchical-gamma-0.1.json'),
    ('robertuito-hier-zero', '../../evaluations/hate_speech/task_b/robertuito-hierarchical-gamma-0.json'),
    ]:
    with open(path) as f:
        evaluations[model_name] = json.load(f)


for key, evals in evaluations.items():
    print(f"We have {len(evals['evaluations']['hate_speech'])} {key} evaluations")



dfs = []
for model_name, model_results in evaluations.items():
    model_evaluations = model_results["evaluations"]["hate_speech"]
    
    if not model_evaluations:
        continue
    
    df = pd.DataFrame(model_evaluations)

    df.columns = [x.split("_", 1)[1] if "_" in x else x for x in df.columns]
    
    mean_df = pd.DataFrame({
        f"{model_name} mean": df.mean(), 
        f"{model_name} std": df.std()
    })
    dfs.append(mean_df)

result_df = pd.concat(dfs, axis=1)
index = [
    'hs_f1', 'hs_precision', 'hs_recall', 'tr_f1',
    'ag_f1', 'macro_hs_f1_score',
    'emr_no_gating', 'emr', 
    'macro_f1',
]
result_df.loc[index]

We have 10 beto evaluations
We have 10 beto-hier evaluations
We have 10 beto-hier-zero evaluations
We have 10 robertuito evaluations
We have 10 robertuito-hier evaluations
We have 10 robertuito-hier-zero evaluations


,beto mean,beto std,beto-hier mean,beto-hier std,beto-hier-zero mean,beto-hier-zero std,robertuito mean,robertuito std,robertuito-hier mean,robertuito-hier std,robertuito-hier-zero mean,robertuito-hier-zero std
hs_f1,0.741171,0.013294,0.734925,0.013436,0.732883,0.008566,0.765693,0.022402,0.759455,0.021526,0.770540,0.011218
hs_precision,0.712582,0.041683,0.743658,0.020801,0.706874,0.026400,0.761301,0.015445,0.749329,0.025522,0.754024,0.021526
hs_recall,0.777879,0.054317,0.727576,0.030116,0.763182,0.034867,0.772424,0.049858,0.773939,0.058314,0.790303,0.044100
tr_f1,0.765226,0.012095,0.758053,0.014196,0.754049,0.010771,0.792220,0.015372,0.794922,0.010608,0.803026,0.010798
ag_f1,0.687683,0.014856,0.674417,0.013812,0.669150,0.017560,0.697813,0.014652,0.680564,0.009469,0.689322,0.021398
macro_hs_f1_score,0.771392,0.014996,0.776030,0.010041,0.765748,0.008783,0.799745,0.013830,0.792895,0.012283,0.801264,0.005334
emr_no_gating,0.684063,0.025306,0.673875,0.016543,0.354125,0.115530,0.709688,0.009684,0.692187,0.009971,0.396000,0.057943
emr,0.684812,0.025478,0.703313,0.011592,0.684625,0.012438,0.709937,0.009822,0.705938,0.011072,0.715563,0.010555
macro_f1,0.731360,0.011938,0.722465,0.009771,0.718694,0.006647,0.751909,0.012441,0.744980,0.009552,0.754296,0.012657


In [119]:
import scipy.stats

hier_emr = [evaluation["eval_emr"] for evaluation in evaluations["beto-hier"]["evaluations"]["hate_speech"]]
standard_emr = [evaluation["eval_emr"] for evaluation in evaluations["beto"]["evaluations"]["hate_speech"]]

scipy.stats.mannwhitneyu(hier_emr, standard_emr, alternative="greater")

MannwhitneyuResult(statistic=73.5, pvalue=0.04093431186532218)

In [120]:
import scipy.stats

hier_emr = [evaluation["eval_emr"] for evaluation in evaluations["robertuito-hier-zero"]["evaluations"]["hate_speech"]]
standard_emr = [evaluation["eval_emr"] for evaluation in evaluations["robertuito"]["evaluations"]["hate_speech"]]

scipy.stats.mannwhitneyu(hier_emr, standard_emr, alternative="greater")

MannwhitneyuResult(statistic=68.0, pvalue=0.09269007167775994)